In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#from fsspec.registry import known_implementations
#from fs import open_fs
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [3]:
spotify_df = pd.read_csv('../PROJECT_3/tracks.csv')
spotify_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


In [4]:
#  Remove features that create noise and result in improved model
removed_df = spotify_df.drop(columns=['id', 'duration_ms', 'id_artists', 'key', 'mode', 'time_signature','valence', 'tempo','energy', 'liveness'])
# Drop the null columns where all values are null
spotify_df = removed_df.dropna(axis='columns', how='all')
# Drop the null rows
cleaned_df = removed_df.dropna()
cleaned_df.describe()

,popularity,explicit,danceability,loudness,speechiness,acousticness,instrumentalness
count,586601.000000,586601.000000,586601.000000,586601.000000,586601.000000,586601.000000,586601.000000
mean,27.573212,0.044091,0.563612,-10.205789,0.104870,0.449803,0.113425
std,18.369417,0.205298,0.166101,5.089422,0.179902,0.348812,0.266843
min,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000
25%,13.000000,0.000000,0.453000,-12.891000,0.034000,0.096900,0.000000
50%,27.000000,0.000000,0.577000,-9.242000,0.044300,0.422000,0.000024
75%,41.000000,0.000000,0.686000,-6.481000,0.076300,0.784000,0.009550
max,100.000000,1.000000,0.991000,5.376000,0.971000,0.996000,1.000000


In [6]:
#Review the correlation of popularity to the other attributes
#popularity is our independent variable for regression (x-value)
pop_corr = cleaned_df.corr()["popularity"]

In [7]:
#This provides the r-value of correlation
pop_corr

popularity          1.000000
explicit            0.211749
danceability        0.186878
loudness            0.327001
speechiness        -0.047415
acousticness       -0.370723
instrumentalness   -0.236403
Name: popularity, dtype: float64

In [8]:
#This computes the r-squared value. 
#note: in this example, the higher r-squared value would indicate that the higher the acousticness level, the less popular the song is
pop_r2 = pop_corr*pop_corr

In [9]:
pop_r2

popularity          1.000000
explicit            0.044838
danceability        0.034923
loudness            0.106930
speechiness         0.002248
acousticness        0.137436
instrumentalness    0.055886
Name: popularity, dtype: float64